## Sentinel-2 Multi-Spectral Imager (MSI) Demo

In [ ]:
# Ignore warning and debug messages
# NB: If things go wrong, you can turn the debugger on and turn the warnings filter off.
import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")
from satpy.utils import debug_off; debug_off()


from satpy import find_files_and_readers

path_msi = './data/msi/'
files = find_files_and_readers(reader='msi_safe',
                               base_dir=path_msi)

# Check the list of files
files

In [ ]:
from satpy import Scene


scn = Scene(files)
scn.available_dataset_names()

In [ ]:
scn.available_composite_names()

In [ ]:
scn.load(scn.available_dataset_names())

In [ ]:
from pyresample import create_area_def

# extent = [lonmin, lonmax, latmin, latmax]
extent = [158.6, 159.2, -54.8, -54.4]
area_def = create_area_def('Macquarie Island', "+proj=eqc", units="degrees",
                           area_extent=(extent[0], extent[2], extent[1], extent[3]), resolution=0.0001)

composite_name = 'true_color'
scn.load([composite_name])
lcn = scn.resample(area_def)
# lcn = scn.resample(resample='native')
lcn.show(composite_name)

In [ ]:
# Get the RED and NIR data arrays
RED = lcn['B04'].values
NIR = lcn['B08'].values
# Compute NDVI
NDVI = (NIR-RED) / (NIR+RED)

In [ ]:
# Plot the data
%matplotlib widget
import matplotlib.pyplot as plt

plt.imshow(NDVI, cmap='YlGn', vmin=0, vmax=1)
plt.show()
